In [1]:
import pandas as pd
import os
import numpy as np

def merge_raw_data(dir):
    dfs = []
    FEATURES = ["accX1", "accY1", "accZ1", "gyroX1", "gyroY1", "gyroZ1"]
    for f in os.scandir(dir):
        df = pd.read_csv(f.path, header=None, names=FEATURES)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

C:\Users\ianng\AppData\Local\Temp\ipykernel_25436\605671945.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from scipy.stats import iqr, skew, kurtosis

def generate_features(merged_df):
    def rms(x):
        return np.sqrt(np.mean(x**2))
    
    WINDOW_SIZE = 25
    transformed_df = merged_df.groupby(np.arange(len(merged_df)) // WINDOW_SIZE).agg({
        "accX1": ['median', iqr, 'var', skew, kurtosis, rms],
        "accY1": ['median', iqr, 'var', skew, kurtosis, rms],
        "accZ1": ['median', iqr, 'var', skew, kurtosis, rms],
        "gyroX1": ['median', iqr, 'var', skew, kurtosis, rms],
        "gyroY1": ['median', iqr, 'var', skew, kurtosis, rms],
        "gyroZ1": ['median', iqr, 'var', skew, kurtosis, rms],
    })
    transformed_df.columns = ['_'.join(col).strip() for col in transformed_df.columns.values]
    return transformed_df

In [3]:
import json

with open('./class_dict.json', 'r') as f:
    class_dict = json.load(f)

In [4]:
class_dict

{'captain': 0,
 'grenade': 1,
 'hulk': 2,
 'ironman': 3,
 'logout': 4,
 'random': 5,
 'reload': 6,
 'shangchi': 7,
 'shield': 8}

In [6]:
RAW_DATA_DIR = './data/handonly/raw/'
MERGED_DATA_DIR = './data/handonly/merged/'
AUGMENTED_DATA_DIR = './data/handonly/augmented/'

os.makedirs(MERGED_DATA_DIR, exist_ok=True)
os.makedirs(AUGMENTED_DATA_DIR, exist_ok=True)
for dir in os.scandir(RAW_DATA_DIR):
    merged_df = merge_raw_data(dir)
    merged_data_file_path = os.path.join(MERGED_DATA_DIR, f"{dir.name}.csv")
    merged_df.to_csv(merged_data_file_path, index=False)
    augmented_df = generate_features(merged_df)
    augmented_df['class'] = class_dict[dir.name]
    augmented_data_file_path = os.path.join(AUGMENTED_DATA_DIR, f"{dir.name}.csv")
    augmented_df.to_csv(augmented_data_file_path, index=False)

In [1]:
import os
import pandas as pd

AUGMENTED_DATA_DIR = './data/handonly/augmented/'
final_dfs = []
for f in os.scandir(AUGMENTED_DATA_DIR):
    df = pd.read_csv(f.path)
    final_dfs.append(df)
final_df = pd.concat(final_dfs, ignore_index=True)

C:\Users\ianng\AppData\Local\Temp\ipykernel_24048\3116978161.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
final_df

,accX1_median,accX1_iqr,accX1_var,accX1_skew,accX1_kurtosis,accX1_rms,accY1_median,accY1_iqr,accY1_var,accY1_skew,...,gyroY1_skew,gyroY1_kurtosis,gyroY1_rms,gyroZ1_median,gyroZ1_iqr,gyroZ1_var,gyroZ1_skew,gyroZ1_kurtosis,gyroZ1_rms,class
0,-1.129,13.216,114.201818,0.177805,-0.231025,11.126271,-19.568,9.984,23.160614,0.323946,...,-0.259142,0.079518,2.011831,0.000,5.752,10.291994,0.053224,-1.136349,3.145395,0
1,-0.748,5.733,91.215854,0.076258,0.821305,9.787008,-19.253,9.911,26.492757,0.396658,...,0.611698,0.076501,2.337397,0.000,4.548,9.479554,0.217057,-1.021267,3.101201,0
2,-1.580,4.452,75.470158,0.256243,0.594493,9.033274,-14.776,9.852,21.830260,-0.023921,...,0.099364,-0.399735,2.349529,0.000,5.916,9.664113,0.079602,-1.167670,3.062429,0
3,-0.945,9.431,71.706618,0.284014,0.222990,8.861287,-19.126,9.807,22.338945,0.300696,...,-0.185722,-0.397327,2.113816,0.000,4.615,9.639234,0.076541,-1.084105,3.069789,0
4,-0.439,6.012,77.837191,0.226591,1.079001,8.837422,-19.568,9.743,22.360077,0.405309,...,0.234392,-0.617752,2.685270,0.000,4.163,8.662564,0.260731,-0.767645,2.960994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3222,-5.716,4.739,13.972839,0.414276,-0.439358,6.315681,-8.731,9.516,37.620856,0.208786,...,-0.174970,-0.419809,1.226451,0.320,3.964,5.501724,-0.385188,-1.168343,2.326235,8
3223,-3.437,5.662,10.418744,-0.020254,-1.057278,5.074997,-8.630,9.567,37.163111,0.156733,...,0.505929,0.285439,1.694564,-0.314,2.764,4.182741,-0.223080,-0.969041,2.091879,8
3224,-5.357,4.889,11.801173,0.332023,-0.376519,6.048951,-8.575,10.316,37.154241,0.438014,...,-0.018013,-0.690893,1.730691,0.000,3.527,4.971032,-0.511169,-1.074111,2.213584,8
3225,-6.720,5.728,15.104921,0.279504,-0.738392,7.340290,-8.803,12.452,41.538592,0.218430,...,0.008010,-0.153432,1.673587,0.000,3.466,5.035948,-0.398909,-0.906319,2.201829,8


In [3]:
final_df = final_df.dropna()

In [4]:
# Should not print anything, indicating no missing values
missing_data_counts = final_df.isna().sum()
for column, count in missing_data_counts.items():
    if count > 0:
        print(f"{column}: {count}")

In [5]:
y = final_df['class'].to_numpy()
x = final_df.drop(['class'], axis=1).to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

# Stratified sampling to get good ratio of classes
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 300]
 [  1 294]
 [  2 300]
 [  3 301]
 [  4 184]
 [  5 302]
 [  6 300]
 [  7 300]
 [  8 297]]


In [8]:
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0 75]
 [ 1 74]
 [ 2 75]
 [ 3 76]
 [ 4 46]
 [ 5 75]
 [ 6 75]
 [ 7 75]
 [ 8 74]]


In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader

x_train = torch.from_numpy(x_train).to(torch.float32)
x_test = torch.from_numpy(x_test).to(torch.float32)
y_train = torch.from_numpy(y_train).to(torch.long)
y_test = torch.from_numpy(y_test).to(torch.long)

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [10]:
import torch
import torch.nn as nn

torch.manual_seed(42)
class Model(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.2):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3 = nn.Linear(hidden_size2, output_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

In [100]:
INPUT_SIZE = x_train.shape[1]
HIDDEN_SIZE1 = 128
HIDDEN_SIZE2 = 64
OUTPUT_SIZE = 9
DROPOUT_PROB = 0.2

model = Model(INPUT_SIZE, HIDDEN_SIZE1, HIDDEN_SIZE2, OUTPUT_SIZE, DROPOUT_PROB)

In [101]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time

NUM_EPOCHS = 20
LEARNING_RATE = 0.005

def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total

    return train_loss, train_accuracy

def evaluate(model, x_test, y_test):
    start_time = time.time()
    print(f'Evaluating model performance for test set of size {len(x_test)}')
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test)
        _, y_pred = torch.max(y_pred, 1)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    print(f'Accuracy: {100. * accuracy_score(y_test, y_pred)}%')
    print(f'Confusion matrix:\n {confusion_matrix(y_test, y_pred)}')
    print(f'Classification report:\n {classification_report(y_test, y_pred)}')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion)
    print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

Epoch 1/20, Train Loss: 1.4250, Train Accuracy: 51.36%
Epoch 2/20, Train Loss: 0.6872, Train Accuracy: 74.52%
Epoch 3/20, Train Loss: 0.4519, Train Accuracy: 84.41%
Epoch 4/20, Train Loss: 0.2817, Train Accuracy: 90.88%
Epoch 5/20, Train Loss: 0.2475, Train Accuracy: 92.71%
Epoch 6/20, Train Loss: 0.1914, Train Accuracy: 94.18%
Epoch 7/20, Train Loss: 0.1612, Train Accuracy: 95.31%
Epoch 8/20, Train Loss: 0.1545, Train Accuracy: 95.73%
Epoch 9/20, Train Loss: 0.1463, Train Accuracy: 95.07%
Epoch 10/20, Train Loss: 0.1242, Train Accuracy: 96.20%
Epoch 11/20, Train Loss: 0.1203, Train Accuracy: 96.39%
Epoch 12/20, Train Loss: 0.1259, Train Accuracy: 96.59%
Epoch 13/20, Train Loss: 0.1033, Train Accuracy: 97.25%
Epoch 14/20, Train Loss: 0.0845, Train Accuracy: 97.09%
Epoch 15/20, Train Loss: 0.0942, Train Accuracy: 97.36%
Epoch 16/20, Train Loss: 0.0741, Train Accuracy: 97.83%
Epoch 17/20, Train Loss: 0.0631, Train Accuracy: 97.94%
Epoch 18/20, Train Loss: 0.1111, Train Accuracy: 96.74%
E

In [102]:
evaluate(model, x_test, y_test)

Evaluating model performance for test set of size 645
Execution time: 0.0009999275207519531 seconds
Accuracy: 99.06976744186046%
Confusion matrix:
 [[74  0  0  1  0  0  0  0  0]
 [ 0 74  0  0  0  0  0  0  0]
 [ 0  1 72  0  0  1  1  0  0]
 [ 0  0  0 76  0  0  0  0  0]
 [ 0  0  0  0 46  0  0  0  0]
 [ 0  1  0  0  0 74  0  0  0]
 [ 0  0  0  0  0  0 75  0  0]
 [ 0  0  0  1  0  0  0 74  0]
 [ 0  0  0  0  0  0  0  0 74]]
Classification report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99        75
           1       0.97      1.00      0.99        74
           2       1.00      0.96      0.98        75
           3       0.97      1.00      0.99        76
           4       1.00      1.00      1.00        46
           5       0.99      0.99      0.99        75
           6       0.99      1.00      0.99        75
           7       1.00      0.99      0.99        75
           8       1.00      1.00      1.00        74

    accuracy   

In [105]:
import os
from datetime import datetime


os.makedirs('models', exist_ok=True)
dt_string = datetime.now().strftime("%d_%m_%Y-%H_%M_%S")
torch.save(model.state_dict(), f'models/mlp_handonly_{dt_string}.pt')

In [106]:
model_params_filename = f'models/mlp_handonly_{dt_string}_params.txt'
with open(model_params_filename, 'w+') as file:
    for name, param in model.named_parameters():
        file.write(f'Layer: {name}\n')
        # Convert the parameter values to a NumPy array
        param_array = param.data.cpu().numpy()
        
        if "weight" in name:
            for row in param_array:
                file.write('{')
                for val in row:
                    file.write(f'%.6f, ' % val)
                file.write('},')
                file.write('\n')
        else: # bias
            file.write('{')
            for val in param_array:
                file.write(f'%.6f, ' % val)
            file.write('},')
            file.write('\n')
        file.write('\n')

In [103]:
for i in range(5):
    print(x_test[i])
    print(y_test[i])

tensor([-1.9230e+00,  5.6460e+00,  7.4045e+01, -1.0269e+00,  1.3948e-01,
         8.9051e+00, -1.0098e+01,  1.0106e+01,  3.3109e+01, -1.0904e-01,
        -1.5151e+00,  1.4467e+01, -1.1136e+01,  1.0010e+01,  4.5010e+01,
        -9.4215e-01, -5.4542e-01,  1.5967e+01,  0.0000e+00,  4.5120e+00,
         8.8519e+00, -1.2393e-02, -9.7918e-01,  2.9164e+00, -3.0330e+00,
         4.1010e+00,  4.2187e+00,  1.5448e-01, -1.7862e+00,  2.8612e+00,
         0.0000e+00,  4.4330e+00,  8.6744e+00, -2.1258e-01, -1.0460e+00,
         2.9698e+00])
tensor(4)
tensor([-1.8440e+00,  4.5260e+00,  1.3106e+01, -1.0284e+00,  2.8936e-02,
         5.2962e+00, -9.6020e+00,  4.9580e+00,  1.3190e+01, -6.1917e-01,
        -4.9850e-01,  1.1990e+01, -6.8280e+00,  1.0802e+01,  8.1115e+01,
        -1.0637e+00, -8.5732e-02,  1.4857e+01,  0.0000e+00,  3.3120e+00,
         8.0135e+00,  7.9283e-02, -8.2036e-01,  2.7881e+00,  0.0000e+00,
         2.9800e-01,  2.1409e+00, -3.0429e-01,  4.4660e+00,  1.4407e+00,
         0.0000e+00

In [104]:
import json

x_test = x_test.tolist()
y_test = y_test.tolist()
labelled_test_data = [{"x": x, "y": y} for x, y in list(zip(x_test, y_test))]
with open('./data/handonly/test_data.json', 'w+') as test_data_json:
    json.dump(labelled_test_data, test_data_json, indent=4)